<a href="https://colab.research.google.com/github/samula98/proj6_v00/blob/main/proj6_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 6: Image Classification with Deep Learning


## Goals

Made a model that will be able to determine whether an image contains a cat or a dog.

## Methods

Build training and testing X and y data sets



# Images



Prior to starting this problem, be sure to enable the GPU runtime processing in your Jupyter notebook.



## Problem Definition

* Write a concise problem definition for the project. Put it in a text field at the top of your Jupyter notebook.

* Load necessary packages.


In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Data Collection



* Load pickled data from X.pickle and y.pickle from the AWS S3 bucket.



* X.pickle

In [ ]:
url_X = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/X.pickle'

In [ ]:
!curl -O {url_X}

In [ ]:
data_X = pd.read_pickle(url_X)
data_X

In [ ]:
data_X.shape

In [ ]:
type(data_X)

* y.pickle

In [ ]:
url_y = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/y.pickle'

In [ ]:
!curl -O {url_y}

In [ ]:
data_y = pd.read_pickle(url_y)
data_y

In [ ]:
type(data_y)

In [ ]:
len(data_y)

* test image of a dog

In [ ]:
url_test_image = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/dog.jpg'

## Data Cleaning



* Scale the values in X so that they fall between 0 and 1 by dividing by 255.



In [ ]:
data_Xscaled = data_X / 255
data_Xscaled

## Exploratory Data Analysis



* Look at the shape of X and y. Ensure that X is 4 dimensional.

In [ ]:
data_Xscaled.shape # X (& X scaled) have 4 dimensions

In [ ]:
data_X.shape

In [ ]:
# sence data_y is a list we can't do the shape method, but we can get the length
len(data_y)

* Plot a few ( >5 ) of the images in X using plt.imshow().

In [ ]:
plt.imshow(data_Xscaled[0])

In [ ]:
plt.imshow(data_Xscaled[2])

In [ ]:
plt.imshow(data_Xscaled[4])

In [ ]:
plt.imshow(data_Xscaled[6])

In [ ]:
plt.imshow(data_Xscaled[8])

In [ ]:
plt.imshow(data_Xscaled[10])

* Look at the response values in y for those images.

In [ ]:
data_y[0:11:2] # looks like dogs == 0 and cats == 1

* Hint: you may want to start with a random subset to get familiar with the process of building a NN.  Then go through the process again with the full set.

## Data Processing


* making testing sets

In [ ]:
# making training set for data_X (first 200 inputs in array)
X = data_Xscaled[:200]
X

In [ ]:
len(X)

In [ ]:
# making training set for data_y (first 200 inputs in list)
y = data_y[:200]
y

In [ ]:
len(y)

* Split X and y into training and testing sets.

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 13)

In [ ]:
X_train.shape, X_test.shape, len(y_train), len(y_test)

In [ ]:
# turn y into numpy array (keeping the same names: y_train & y_test)
# probably not the best practice but it makes my life easy
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print(type(y_train))
print(type(y_test))

In [ ]:
# looking at shape for the train test split, now all 4 of them are np arrays
X_train.shape, X_test.shape, y_train.shape, y_test.shape


*  Build a convolutional neural network with the following:
  * Sequential layers
  * At least two 2D convolutional layers using the 'relu' activation function and a (3,3) kernel size.
  * A MaxPooling2D layer after each 2D convolutional layer that has a pool size of (2,2).
  * A dense output layer using the 'sigmoid' activation function.
  Note: you can play around with the number of layers and nodes to try to get better performance.


In [ ]:
model = Sequential()

In [ ]:
model.add(Flatten())

In [ ]:
# difining the first hidden layer (hidden1)
model.add(
    Dense(
        name = 'hidden1',
        units = 144,
        activation = 'relu',
    )
)

In [ ]:
# defining the second hidden layer (hidden2)
model.add(
    Dense(
        name = 'hidden2',
        units = 144,
        activation = 'relu',
    )
)

In [ ]:
# defining the output layer
model.add(
    Dense(
        name = 'output',
        units = 10,
        activation = 'softmax',
    )
)


* Compile your model. Use the 'adam' optimizer. Determine which loss function and metric is most appropriate for this problem.


In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)


* Fit your model using the training set.


In [ ]:
model.fit(X_train, y_train, epochs = 8) ;

In [ ]:
model.summary()

In [ ]:
# visualizing the model
plot_model(model)


* Evaluate your model using the testing set.


In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {val_accuracy}')
print(f'Loss: {val_loss}')


* Plot the distribution of probabilities for the testing set.



* Define a function that will read in a new image and convert it to a 4 dimensional array of pixels (ask the instructor for help with this). Hint: [numpy.reshape]( https://numpy.org/doc/stable/reference/generated/numpy.reshape.html )


In [ ]:
# from Wil:
%%capture output
%%bash
apt-get update
apt-get install -y exiftool

In [ ]:
%%capture output
!pip install pandas boto3

In [ ]:
import glob
import os
import re
import json
import cv2
import requests
import io
import geopandas as gpd
import folium
import pprint
from PIL import Image
from shapely.geometry import Point

In [ ]:
def image4D (url):
  resp = requests.get(url, stream = True)
  img = Image.open(io.BytesIO(resp.content)).convert('L')
  img = np.array(img)
  return img

In [ ]:
puppy = image4D(url_test_image)

In [ ]:
resize_image = cv2.resize(puppy, (50,50))

In [ ]:
resize_image = resize_image[np.newaxis, ...]
final = resize_image.reshape(resize_image.shape + (1,))

In [ ]:
print(final.shape)

In [ ]:
plt.imshow(final[0])


* Use the function defined above to read in the dog.jpg image that is saved in the AWS S3 bucket.


In [ ]:
model.predict(final)

In [ ]:
plt.imshow(final[0])

predictions = model.predict(final)


* Use the neural network you created to predict whether the image is a dog or a cat.



## Communication of Results



* Communicate the results of your analysis.



## **BONUS** (optional)



* Upload an image of your (or your friend's or family's) dog or cat and use your model to predict whether the image is a dog or cat.
* Hint: you'll probably need to convert the image from color to grayscale.  OpenCV, pillow, and other libraries are your friend.